In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation,BatchNormalization 
from keras.utils import np_utils

from sklearn import preprocessing 

import warnings
warnings.filterwarnings('ignore')


/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train=pd.read_csv('/root/Downloads/Titannic/train.csv')
test=pd.read_csv('/root/Downloads/Titannic/test.csv')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
ID = 'PassengerId' 
TARGET = 'Survived' 
NAME = 'Name'
TICKET = 'Ticket'
CABIN = 'Cabin'
FEATURES = train.columns.drop([ID, TARGET, NAME]) 

In [5]:
print FEATURES

Index([u'Pclass', u'Sex', u'Age', u'SibSp', u'Parch', u'Ticket', u'Fare',
       u'Cabin', u'Embarked'],
      dtype='object')


In [6]:
train_y = train[TARGET].values

In [7]:
train = train.drop('Survived',axis = 1)
train = train.drop('Name', axis = 1)
test = test.drop('Name', axis = 1)

In [8]:
all_data = pd.concat([train, test], axis = 0)

In [9]:
f_cat = all_data[FEATURES].select_dtypes(include=['object']).columns
f_num = all_data[FEATURES].select_dtypes(exclude=['object']).columns

In [10]:
all_data[f_num] = all_data[f_num].fillna(all_data[f_num].mean())
all_data[f_cat] = all_data[f_cat].fillna('?')

In [11]:
dummy_cat = pd.get_dummies(all_data[f_cat])

In [12]:
all_data = pd.DataFrame(preprocessing.scale(all_data[f_num]), columns=f_num)

In [13]:
dummy_cat.index = all_data.index
all_data = pd.concat([all_data, dummy_cat], axis=1, ignore_index=True)

In [14]:
X = all_data.values
x_train = X[:891, 0:]
y_train = np_utils.to_categorical(train_y,2)
print len(x_train)
print len(y_train)

891
891


In [15]:
def create_nn_model(input_dim, activation, layers, optimizer):
    model = Sequential()
    l_num = 0
    for l in layers:
        if l_num == 0:
            model.add(Dense(l, input_dim = input_dim, activation = activation, init = 'he_normal'))
        else: 
            model.add(Dense(l, activation = activation, init = 'he_normal')) 
        l_num = l_num + 1 
    model.add(BatchNormalization(epsilon=1e-6,axis=1))
    model.add(Dropout(0.25))
    model.add(Dense(10))
    model.add(Activation('relu'))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy']) 
    return model

In [16]:
model = create_nn_model(input_dim = X.shape[1], activation = 'selu', layers = [1500, 500,  50], optimizer = 'adagrad') 
fit = model.fit(x_train, y_train, batch_size=50, nb_epoch=50, validation_split=0.1, verbose = 1) 
model.summary()

Train on 801 samples, validate on 90 samples
Epoch 1/50
801/801 [==============================] - 1s 1ms/step - loss: 0.9081 - acc: 0.6617 - val_loss: 0.5680 - val_acc: 0.8111
Epoch 2/50
801/801 [==============================] - 1s 944us/step - loss: 0.4222 - acc: 0.8352 - val_loss: 0.4572 - val_acc: 0.8556
Epoch 3/50
801/801 [==============================] - 1s 958us/step - loss: 0.3330 - acc: 0.8839 - val_loss: 0.4297 - val_acc: 0.8556
Epoch 4/50
801/801 [==============================] - 1s 954us/step - loss: 0.2487 - acc: 0.9051 - val_loss: 0.3772 - val_acc: 0.8333
Epoch 5/50
801/801 [==============================] - 1s 1ms/step - loss: 0.1590 - acc: 0.9463 - val_loss: 0.4219 - val_acc: 0.8444
Epoch 6/50
801/801 [==============================] - 1s 947us/step - loss: 0.1253 - acc: 0.9625 - val_loss: 0.5518 - val_acc: 0.8222
Epoch 7/50
801/801 [==============================] - 1s 1ms/step - loss: 0.0968 - acc: 0.9788 - val_loss: 0.4155 - val_acc: 0.8667
Epoch 8/50
801/801 [===

In [17]:
x_test = X[891:1309, 0:]

In [18]:
result = model.predict_classes(x_test,batch_size = 50 , verbose=1)
df=pd.DataFrame({ 'PassengerId': range(892, 1310), #[x for x in range(892,1310)],
                   'Survived': result })
df.to_csv('/root/Downloads/Titannic/result.csv',index=False)

418/418 [==============================] - 0s 250us/step
